<a href="https://colab.research.google.com/github/xida2020/element_tet_p3/blob/main/zhou_adaptive_tet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solve Laplace eigenvalue problem on the uniform meshes
$-\frac12\Delta u = \lambda u $, in $\Omega = (-5,5)^3$
$\backslash(0,5)^3$

and
$u = 0$, on $\partial\Omega.$

The first eigenvalue $\lambda_1\approx0.2106412625$

# Install scikit-fem:

In [8]:
!pip install scikit-fem[all]

# element_tet_p3

In [3]:
import numpy as np

from skfem.element.element_h1 import ElementH1
from skfem.refdom import RefTet


class ElementTetP3(ElementH1):
    """Piecewise cubic element."""

    nodal_dofs = 1
    facet_dofs = 1
    edge_dofs = 2
    maxdeg = 3
    dofnames = ["u", "u", "u", "u"]
    doflocs = np.array([[0., 0., 0.], #nodal O  ==0
                        [1., 0., 0.], #nodal x  ==1
                        [0., 1., 0.], #nodal y  ==2
                        [0., 0., 1.], #nodal z  ==3
                        [1./ 3, 0., 0.], #edge O->x  ==4
                        [2./ 3, 0., 0.], #edge O->x  ==5
                        [2./ 3, 1./ 3, 0.], #edge x->y  ==6
                        [1./ 3, 2./ 3, 0.], #edge x->y  ==7
                        [0., 1./ 3, 0.], #edge O->y  ==8
                        [0., 2./ 3, 0.], #edge O->y  ==9
                        [0., 0., 1./ 3], #edge O->z  ==10
                        [0., 0., 2./ 3], #edge O->z  ==11
                        [2./ 3, 0., 1./ 3], #edge x->z  ==12
                        [1./ 3, 0., 2./ 3], #edge x->z  ==13
                        [0., 2./ 3, 1./ 3], #edge y->z  ==14
                        [0., 1./ 3, 2./ 3], #edge y->z  ==15
                        [1./ 3, 1./ 3, 0.], #facet Oxy  ==16
                        [1./ 3, 0., 1./ 3], #facet Oxz  ==17
                        [0., 1./ 3, 1./ 3], #facet Oyz  ==18
                        [1./ 3, 1./ 3, 1./ 3]]) #facet xyz  ==19
    refdom = RefTet

    def lbasis(self, X, i):
        x, y, z = X

        if i == 0:  # at (0,0,0)
            phi = -(3.*x + 3.*y + 3.*z - 1.)*(3.*x + 3.*y + 3.*z - 2.)*(x/2 + y/2 + z/2 - 1./2)
            dphi = np.array([
                18.*x + 18.*y + 18.*z - 27.*x*y - 27.*x*z - 27.*y*z 
                - (27.*x**2)/2 - (27.*y**2)/2 - (27.*z**2)/2 - 11./2,
                18.*x + 18.*y + 18.*z - 27.*x*y - 27.*x*z - 27.*y*z 
                - (27.*x**2)/2 - (27.*y**2)/2 - (27.*z**2)/2 - 11./2,
                18.*x + 18.*y + 18.*z - 27.*x*y - 27.*x*z - 27.*y*z 
                - (27.*x**2)/2 - (27.*y**2)/2 - (27.*z**2)/2 - 11./2
            ])
        elif i == 1:  # at (1,0,0)
            phi = (x*(3.*x - 1.)*(3.*x - 2))/2
            dphi = np.array([
                (27.*x**2)/2 - 9.*x + 1.,
                0*y,
                0*z
            ])
        elif i == 2:  # at (0,1,0)
            phi = (y*(3.*y - 1.)*(3.*y - 2.))/2
            dphi = np.array([
                0*x,
                (27.*y**2)/2 - 9.*y + 1.,
                0*z
            ])
        elif i == 3:  # at (0, 0, 1)
            phi = (z*(3.*z - 1.)*(3.*z - 2.))/2
            dphi = np.array([
                0*x,
                0*x,
                (27.*z**2)/2 - 9.*z + 1.
            ])
        elif i == 4:  # at (1/3, 0, 0)
            phi = x*(3.*x + 3.*y + 3.*z - 2.)*((9.*x)/2 + (9.*y)/2 + (9.*z)/2 - 9./2)
            dphi = np.array([
                (81.*x**2)/2 + 54.*x*y + 54.*x*z - 45.*x + (27.*y**2)/2 
                + 27.*y*z - (45.*y)/2 + (27.*z**2)/2 - (45.*z)/2 + 9.,
                (9.*x*(6.*x + 6.*y + 6.*z - 5.))/2,
                (9.*x*(6.*x + 6.*y + 6.*z - 5.))/2
            ])
        elif i == 5:  # at (2/3, 0, 0)
            phi = -(9.*x*(3.*x - 1.)*(x + y + z - 1.))/2
            dphi = np.array([
                36.*x + (9.*y)/2 + (9.*z)/2 - 27.*x*y - 27.*x*z - (81.*x**2)/2 - 9./2,
                -(9.*x*(3.*x - 1.))/2,
                -(9.*x*(3.*x - 1.))/2
            ])
        elif i == 6:  # at (2/3, 1/3, 0)
            phi = (9.*x*y*(3.*x - 1.))/2
            dphi = np.array([
                (9.*y*(6.*x - 1.))/2,
                (9.*x*(3.*x - 1.))/2,
                0*z
            ])
        elif i == 7:  # at (1/3, 2/3, 0)
            phi = (9.*x*y*(3.*y - 1.))/2
            dphi = np.array([
                (9.*y*(3.*y - 1.))/2,
                (9.*x*(6.*y - 1.))/2,
                0*z
            ])
        elif i == 8:  # at (0, 1/3, 0)
            phi = y*(3.*x + 3.*y + 3.*z - 2.)*((9.*x)/2 + (9.*y)/2 + (9.*z)/2 - 9./2)
            dphi = np.array([
                (9.*y*(6.*x + 6.*y + 6.*z - 5.))/2,
                (27.*x**2)/2 + 54.*x*y + 27.*x*z - (45.*x)/2 + (81.*y**2)/2 
                + 54.*y*z - 45.*y + (27.*z**2)/2 - (45.*z)/2 + 9.,
                (9.*y*(6.*x + 6.*y + 6.*z - 5.))/2
            ])
        elif i == 9:  # at (0, 2/3, 0)
            phi = -(9.*y*(3.*y - 1.)*(x + y + z - 1.))/2
            dphi = np.array([
                -(9.*y*(3.*y - 1.))/2,
                (9.*x)/2 + 36.*y + (9.*z)/2 - 27.*x*y - 27.*y*z - (81.*y**2)/2 - 9./2,
                -(9.*y*(3.*y - 1.))/2
            ])
        elif i == 10:  # at (0, 0, 1/3)
            phi = z*(3.*x + 3.*y + 3.*z - 2.)*((9.*x)/2 + (9.*y)/2 + (9.*z)/2 - 9./2)
            dphi = np.array([
                (9.*z*(6.*x + 6.*y + 6.*z - 5.))/2,
                (9.*z*(6.*x + 6.*y + 6.*z - 5.))/2,
                (27.*x**2)/2 + 27.*x*y + 54.*x*z - (45.*x)/2 
                + (27.*y**2)/2 + 54.*y*z - (45.*y)/2 + (81.*z**2)/2 - 45.*z + 9.
            ])
        elif i == 11:  # at (0, 0, 2/3)
            phi = -(9.*z*(3.*z - 1.)*(x + y + z - 1.))/2
            dphi = np.array([
                -(9.*z*(3.*z - 1.))/2,
                -(9.*z*(3.*z - 1.))/2,
                (9.*x)/2 + (9.*y)/2 + 36.*z - 27.*x*z - 27.*y*z 
                - (81.*z**2)/2 - 9./2
            ])
        elif i == 12:  # at (2/3, 0, 1/3)
            phi = (9.*x*z*(3.*x - 1.))/2
            dphi = np.array([
                (9.*z*(6.*x - 1.))/2,
                0*y,
                (9.*x*(3.*x - 1.))/2
            ])
        elif i == 13:  # at (1/3, 0.0, 2/3)
            phi = (9.*x*z*(3.*z - 1.))/2
            dphi = np.array([
                (9.*z*(3.*z - 1.))/2,
                0*y,
                (9.*x*(6.*z - 1.))/2
            ])
        elif i == 14:  # at (0, 2/3, 1/3)
            phi = (9.*y*z*(3.*y - 1.))/2
            dphi = np.array([
                0*x,
                (9.*z*(6.*y - 1.))/2,
                (9.*y*(3.*y - 1.))/2
            ])
        elif i == 15:  # at (0, 1/3, 2/3)
            phi = (9.*y*z*(3.*z - 1.))/2
            dphi = np.array([
                0*x,
                (9.*z*(3.*z - 1.))/2,
                (9.*y*(6.*z - 1.))/2
            ])
        elif i == 16:  # at (1/3, 1/3, 0)
            phi = -x*y*(27.*x + 27.*y + 27.*z - 27.)
            dphi = np.array([
                -27.*y*(2.*x + y + z - 1.),
                -27.*x*(x + 2.*y + z - 1.),
                -27.*x*y
            ])
        elif i == 17:  # at (1/3, 0, 1/3)
            phi = -x*z*(27.*x + 27.*y + 27.*z - 27.)
            dphi = np.array([
                -27.*z*(2.*x + y + z - 1.),
                -27.*x*z,
                -27.*x*(x + y + 2.*z - 1.)
            ])
        elif i == 18:  # at (0, 1/3, 1/3)
            phi = -y*z*(27.*x + 27.*y + 27.*z - 27.)
            dphi = np.array([
                -27.*y*z,
                -27.*z*(x + 2.*y + z - 1.),
                -27.*y*(x + y + 2.*z - 1.)
            ]) 
        elif i == 19:  # at (1/3, 1/3, 1/3)
            phi = 27.*x*y*z
            dphi = np.array([
                27.*y*z,
                27.*x*z,
                27.*x*y
            ])
        else:
            self._index_error()

        return phi, dphi

# solve Laplace eigenvalue problem on uniform meshes
$\Delta u = \lambda u $, in $\Omega = (0,1)^3$
and
$u = 0$, on $\partial\Omega.$
The first eigenvalue $\lambda_1=3\pi^2\approx29.608813203268074$

# Using p3 elements uniformly

In [6]:
from skfem import *
from skfem.models.poisson import laplace, mass
import numpy as np


for itr in range(10):
  p1 = np.linspace(0, 5, itr+2)
  m1 = MeshTet.init_tensor(*(p1,) * 3)
  m2 = m1.mirrored([0,0,1])
  m3 = m1.mirrored([0,1,0])
  m4 = m1.mirrored([1,0,0])
  m5 = m1.mirrored([0,1,1])
  m6 = m1.mirrored([1,0,1])
  m7 = m1.mirrored([1,1,0])
  m8 = m7.mirrored([0,0,1])
  m = m8+m2+m3+m4+m5+m6+m7
  e = ElementTetP3()
  basis = Basis(m, e) 
  K = asm(laplace, basis)
  M = asm(mass, basis)
  D=basis.get_dofs()
  Lh,eigen_uh = solve(*condense(0.5*K, M, D=D), solver=solver_eigen_scipy_sym(sigma=0,k=1))
  Dof= K.shape[0]
  error = np.abs(Lh-0.2106412625)
  print("{},{},{},{}".format(itr,Dof,Lh,error))

KeyboardInterrupt: ignored

# element_tet_p4

In [11]:
import numpy as np

from skfem.element.element_h1 import ElementH1
from skfem.refdom import RefTet


class ElementTetP4(ElementH1):
    """Piecewise qurtic element."""

    nodal_dofs = 1
    edge_dofs = 3
    facet_dofs = 3
    interior_dofs = 1
    maxdeg = 4
    dofnames = ['u', 'u', 'u', 'u', 'u', 'u', 'u','u']
    doflocs = np.array([[0., 0., 0.], #nodal O  ==0
                        [1., 0., 0.], #nodal x  ==1
                        [0., 1., 0.], #nodal y  ==2
                        [0., 0., 1.], #nodal z  ==3
                        [0.25, 0., 0.], #edge O->x  ==4
                        [0.50, 0., 0.], #edge O->x  ==5
                        [0.75, 0., 0.], #edge O->x  ==6
                        [0.75, 0.25, 0.], #edge x->y  ==7
                        [0.50, 0.50, 0.], #edge x->y  ==8
                        [0.25, 0.75, 0.], #edge x->y  ==9
                        [0., 0.25, 0.], #edge O->y  ==10
                        [0., 0.50, 0.], #edge O->y  ==11
                        [0., 0.75, 0.], #edge O->y  ==12
                        [0., 0., 0.25], #edge O->z  ==13
                        [0., 0., 0.50], #edge O->z  ==14
                        [0., 0., 0.75], #edge O->z  ==15
                        [0.75, 0.0, 0.25], #edge x->z  ==16
                        [0.50, 0.0, 0.50], #edge x->z  ==17
                        [0.25, 0.0, 0.75], #edge x->z  ==18
                        [0.0, 0.75, 0.25], #edge y->z  ==19
                        [0.0, 0.50, 0.50], #edge y->z  ==20
                        [0.0, 0.25, 0.75], #edge y->z  ==21
                        [0.25, 0.25, 0.], #facet Oxy  ==22
                        [0.50, 0.25, 0.], #facet Oxy  ==23
                        [0.25, 0.50, 0.], #facet Oxy  ==24
                        [0.25, 0., 0.25], #facet Oxz  ==25
                        [0.50, 0., 0.25], #facet Oxz  ==26
                        [0.25, 0., 0.50], #facet Oxz  ==27
                        [0., 0.25, 0.25], #facet Oyz  ==28
                        [0., 0.50, 0.25], #facet Oyz  ==29
                        [0., 0.25, 0.50], #facet Oyz  ==30
                        [0.50, 0.25, 0.25], #facet xyz  ==31
                        [0.25, 0.50, 0.25], #facet xyz  ==32
                        [0.25, 0.25, 0.50], #facet xyz  ==33
                        [0.25, 0.25, 0.25]]) #center  ==34
    refdom = RefTet

    def lbasis(self, X, i):
        x, y, z = X

        if i == 0:  # at (0,0,0)
            phi = (4*x + 4*y + 4*z - 1)*(4*x + 4*y + 4*z - 2)*(4*x + 4*y + 4*z - 3)*(x/6 + y/6 + z/6 - 1/6)
            dphi = np.array([
                (128*x**3)/3 + 128*x**2*y + 128*x**2*z - 80*x**2 + 128*x*y**2 
                + 256*x*y*z - 160*x*y + 128*x*z**2 - 160*x*z + (140*x)/3 
                + (128*y**3)/3 + 128*y**2*z - 80*y**2 + 128*y*z**2 - 160*y*z 
                + (140*y)/3 + (128*z**3)/3 - 80*z**2 + (140*z)/3 - 25/3,
                (128*x**3)/3 + 128*x**2*y + 128*x**2*z - 80*x**2 + 128*x*y**2 
                + 256*x*y*z - 160*x*y + 128*x*z**2 - 160*x*z + (140*x)/3 
                + (128*y**3)/3 + 128*y**2*z - 80*y**2 + 128*y*z**2 - 160*y*z 
                + (140*y)/3 + (128*z**3)/3 - 80*z**2 + (140*z)/3 - 25/3,
                (128*x**3)/3 + 128*x**2*y + 128*x**2*z - 80*x**2 + 128*x*y**2 
                + 256*x*y*z - 160*x*y + 128*x*z**2 - 160*x*z + (140*x)/3 
                + (128*y**3)/3 + 128*y**2*z - 80*y**2 + 128*y*z**2 - 160*y*z 
                + (140*y)/3 + (128*z**3)/3 - 80*z**2 + (140*z)/3 - 25/3,
            ])
        elif i == 1: # at (1,0,0)
            phi = (x*(4*x - 1)*(4*x - 2)*(4*x - 3))/6
            dphi = np.array([
                (128*x**3)/3 - 48*x**2 + (44*x)/3 - 1,
                0*x,
                0*x,
            ])
        elif i == 2: # at (0,1,0)
            phi = (y*(4*y - 1)*(4*y - 2)*(4*y - 3))/6
            dphi = np.array([
                0*x,
                (128*y**3)/3 - 48*y**2 + (44*y)/3 - 1,
                0*x,
            ])
        elif i == 3: # at (0,0,1)
            phi = (z*(4*z - 1)*(4*z - 2)*(4*z - 3))/6
            dphi = np.array([
                0*x,
                0*x,
                (128*z**3)/3 - 48*z**2 + (44*z)/3 - 1,
            ])
        elif i == 4:
            phi = -x*(4*x + 4*y + 4*z - 2)*(4*x + 4*y + 4*z - 3)*((8*x)/3 + (8*y)/3 + (8*z)/3 - 8/3)
            dphi = np.array([
                - (4*x + 4*y + 4*z - 2)*(4*x + 4*y + 4*z - 3)*((8*x)/3 
                    + (8*y)/3 + (8*z)/3 - 8/3) - (8*x*(4*x + 4*y + 4*z - 2)*(4*x 
                        + 4*y + 4*z - 3))/3 - 4*x*(4*x + 4*y + 4*z - 2)*((8*x)/3 
                    + (8*y)/3 + (8*z)/3 - 8/3) - 4*x*(4*x + 4*y + 4*z - 3)*((8*x)/3 
                    + (8*y)/3 + (8*z)/3 - 8/3),
                -(16*x*(24*x**2 + 48*x*y + 48*x*z - 36*x 
                    + 24*y**2 + 48*y*z - 36*y + 24*z**2 - 36*z + 13))/3,
                -(16*x*(24*x**2 + 48*x*y + 48*x*z - 36*x 
                    + 24*y**2 + 48*y*z - 36*y + 24*z**2 - 36*z + 13))/3
            ])
        elif i == 5:
            phi = x*(4*x - 1)*(4*x + 4*y + 4*z - 3)*(4*x + 4*y + 4*z - 4)
            dphi = np.array([
                (4*x - 1)*(4*x + 4*y + 4*z - 3)*(4*x + 4*y + 4*z - 4) 
                + 4*x*(4*x + 4*y + 4*z - 3)*(4*x + 4*y + 4*z - 4) 
                + 4*x*(4*x - 1)*(4*x + 4*y + 4*z - 3) 
                + 4*x*(4*x - 1)*(4*x + 4*y + 4*z - 4),
                4*x*(4*x - 1)*(8*x + 8*y + 8*z - 7),
                4*x*(4*x - 1)*(8*x + 8*y + 8*z - 7),
            ])
        elif i == 6:
            phi = -x*(4*x - 1)*(4*x - 2)*((8*x)/3 + (8*y)/3 + (8*z)/3 - 8/3)
            dphi = np.array([
                64*x*y - (16*y)/3 - (16*z)/3 - (224*x)/3 
                + 64*x*z - 128*x**2*y - 128*x**2*z + 224*x**2 
                - (512*x**3)/3 + 16/3,
                -(8*x*(4*x - 1)*(4*x - 2))/3,
                -(8*x*(4*x - 1)*(4*x - 2))/3,
            ])
        elif i == 7:
            phi = (8*x*y*(4*x - 1)*(4*x - 2))/3
            dphi = np.array([
                (16*y*(24*x**2 - 12*x + 1))/3,
                (8*x*(4*x - 1)*(4*x - 2))/3,
                0*x,
            ])
        elif i == 8:
            phi = 4*x*y*(4*x - 1)*(4*y - 1)
            dphi = np.array([
                4*y*(8*x - 1)*(4*y - 1),
                4*x*(4*x - 1)*(8*y - 1),
                0*x,
            ])
        elif i == 9:
            phi = (8*x*y*(4*y - 1)*(4*y - 2))/3
            dphi = np.array([
                (8*y*(4*y - 1)*(4*y - 2))/3,
                (16*x*(24*y**2 - 12*y + 1))/3,
                0*x,
            ])
        elif i == 10:
            phi = -y*(4*x + 4*y + 4*z - 2)*(4*x + 4*y + 4*z - 3)*((8*x)/3 
                + (8*y)/3 + (8*z)/3 - 8/3)
            dphi = np.array([
                -(16*y*(24*x**2 + 48*x*y + 48*x*z - 36*x + 24*y**2 + 48*y*z - 36*y 
                    + 24*z**2 - 36*z + 13))/3,
                - (4*x + 4*y + 4*z - 2)*(4*x + 4*y + 4*z - 3)*((8*x)/3 + (8*y)/3 
                    + (8*z)/3 - 8/3) - (8*y*(4*x + 4*y + 4*z - 2)*(4*x + 4*y + 4*z - 3))/3 
                - 4*y*(4*x + 4*y + 4*z - 2)*((8*x)/3 + (8*y)/3 + (8*z)/3 - 8/3) 
                - 4*y*(4*x + 4*y + 4*z - 3)*((8*x)/3 + (8*y)/3 + (8*z)/3 - 8/3),
                -(16*y*(24*x**2 + 48*x*y + 48*x*z - 36*x + 24*y**2 + 48*y*z 
                    - 36*y + 24*z**2 - 36*z + 13))/3,
            ])
        elif i == 11:
            phi = y*(4*y - 1)*(4*x + 4*y + 4*z - 3)*(4*x + 4*y + 4*z - 4)
            dphi = np.array([
                4*y*(4*y - 1)*(8*x + 8*y + 8*z - 7),
                (4*y - 1)*(4*x + 4*y + 4*z - 3)*(4*x + 4*y + 4*z - 4) 
                + 4*y*(4*x + 4*y + 4*z - 3)*(4*x + 4*y + 4*z - 4) 
                + 4*y*(4*y - 1)*(4*x + 4*y + 4*z - 3) 
                + 4*y*(4*y - 1)*(4*x + 4*y + 4*z - 4),
                4*y*(4*y - 1)*(8*x + 8*y + 8*z - 7),
            ])
        elif i == 12:
            phi = -y*(4*y - 1)*(4*y - 2)*((8*x)/3 + (8*y)/3 + (8*z)/3 - 8/3)
            dphi = np.array([
                -(8*y*(4*y - 1)*(4*y - 2))/3,
                64*x*y - (224*y)/3 - (16*z)/3 - (16*x)/3 + 64*y*z 
                - 128*x*y**2 - 128*y**2*z + 224*y**2 - (512*y**3)/3 + 16/3,
                -(8*y*(4*y - 1)*(4*y - 2))/3,
            ])
        elif i == 13:
            phi = -z*(4*x + 4*y + 4*z - 2)*(4*x + 4*y + 4*z - 3)*((8*x)/3 
                + (8*y)/3 + (8*z)/3 - 8/3)
            dphi = np.array([
                -(16*z*(24*x**2 + 48*x*y + 48*x*z - 36*x + 24*y**2 + 48*y*z 
                    - 36*y + 24*z**2 - 36*z + 13))/3,
                -(16*z*(24*x**2 + 48*x*y + 48*x*z - 36*x + 24*y**2 + 48*y*z 
                    - 36*y + 24*z**2 - 36*z + 13))/3,
                - (4*x + 4*y + 4*z - 2)*(4*x + 4*y + 4*z - 3)*((8*x)/3 
                    + (8*y)/3 + (8*z)/3 - 8/3) - (8*z*(4*x + 4*y 
                        + 4*z - 2)*(4*x + 4*y + 4*z - 3))/3 
                    - 4*z*(4*x + 4*y + 4*z - 2)*((8*x)/3 
                        + (8*y)/3 + (8*z)/3 - 8/3) - 4*z*(4*x + 4*y 
                        + 4*z - 3)*((8*x)/3 + (8*y)/3 + (8*z)/3 - 8/3),
            ])
        elif i == 14:
            phi = z*(4*z - 1)*(4*x + 4*y + 4*z - 3)*(4*x + 4*y + 4*z - 4)
            dphi = np.array([                                                                                                                                                
                4*z*(4*z - 1)*(8*x + 8*y + 8*z - 7),
                4*z*(4*z - 1)*(8*x + 8*y + 8*z - 7),
                (4*z - 1)*(4*x + 4*y + 4*z - 3)*(4*x + 4*y + 4*z - 4) 
                + 4*z*(4*x + 4*y + 4*z - 3)*(4*x + 4*y + 4*z - 4) 
                + 4*z*(4*z - 1)*(4*x + 4*y + 4*z - 3) 
                + 4*z*(4*z - 1)*(4*x + 4*y + 4*z - 4),
            ])
        elif i == 15:
            phi = -z*(4*z - 1)*(4*z - 2)*((8*x)/3 + (8*y)/3 + (8*z)/3 - 8/3)
            dphi = np.array([
                -(8*z*(4*z - 1)*(4*z - 2))/3,
                -(8*z*(4*z - 1)*(4*z - 2))/3,
                64*x*z - (16*y)/3 - (224*z)/3 - (16*x)/3 + 64*y*z - 128*x*z**2 
                - 128*y*z**2 + 224*z**2 - (512*z**3)/3 + 16/3,
            ])
        elif i == 16:
            phi = (8*x*z*(4*x - 1)*(4*x - 2))/3
            dphi = np.array([
                (16*z*(24*x**2 - 12*x + 1))/3,
                0*x,
                (8*x*(4*x - 1)*(4*x - 2))/3,
            ])
        elif i == 17:
            phi = 4*x*z*(4*x - 1)*(4*z - 1)
            dphi = np.array([
                4*z*(8*x - 1)*(4*z - 1),
                0*x,
                4*x*(4*x - 1)*(8*z - 1),
            ])
        elif i == 18:
            phi = (8*x*z*(4*z - 1)*(4*z - 2))/3
            dphi = np.array([
                (8*z*(4*z - 1)*(4*z - 2))/3,
                0*x,
                (16*x*(24*z**2 - 12*z + 1))/3,
            ])
        elif i == 19:
            phi = (8*y*z*(4*y - 1)*(4*y - 2))/3
            dphi = np.array([
                0*x,
                (16*z*(24*y**2 - 12*y + 1))/3,
                (8*y*(4*y - 1)*(4*y - 2))/3,
            ])
        elif i == 20:
            phi = 4*y*z*(4*y - 1)*(4*z - 1)
            dphi = np.array([
                0*x,
                4*z*(8*y - 1)*(4*z - 1),
                4*y*(4*y - 1)*(8*z - 1),
            ])
        elif i == 21:
            phi = (8*y*z*(4*z - 1)*(4*z - 2))/3
            dphi = np.array([
                0*x,
                (8*z*(4*z - 1)*(4*z - 2))/3,
                (16*y*(24*z**2 - 12*z + 1))/3,
            ])
        elif i == 22:
            phi = x*y*(4*x + 4*y + 4*z - 3)*(32*x + 32*y + 32*z - 32)
            dphi = np.array([
                32*y*(12*x**2 + 16*x*y + 16*x*z - 14*x + 4*y**2 + 8*y*z 
                    - 7*y + 4*z**2 - 7*z + 3),
                32*x*(4*x**2 + 16*x*y + 8*x*z - 7*x + 12*y**2 + 16*y*z 
                    - 14*y + 4*z**2 - 7*z + 3),
                32*x*y*(8*x + 8*y + 8*z - 7),
            ])
        elif i == 23:
            phi = -x*y*(4*x - 1)*(32*x + 32*y + 32*z - 32)
            dphi = np.array([
                -32*y*(8*x*y - y - z - 10*x + 8*x*z + 12*x**2 + 1),
                -32*x*(4*x - 1)*(x + 2*y + z - 1),
                -32*x*y*(4*x - 1),
            ])
        elif i == 24:
            phi = -x*y*(4*y - 1)*(32*x + 32*y + 32*z - 32)
            dphi = np.array([
                -32*y*(4*y - 1)*(2*x + y + z - 1),
                -32*x*(8*x*y - 10*y - z - x + 8*y*z + 12*y**2 + 1),
                -32*x*y*(4*y - 1),
            ])
        elif i == 25:
            phi = x*z*(4*x + 4*y + 4*z - 3)*(32*x + 32*y + 32*z - 32)
            dphi = np.array([
                32*z*(12*x**2 + 16*x*y + 16*x*z - 14*x + 4*y**2 + 8*y*z 
                    - 7*y + 4*z**2 - 7*z + 3),
                32*x*z*(8*x + 8*y + 8*z - 7),
                32*x*(4*x**2 + 8*x*y + 16*x*z - 7*x + 4*y**2 + 16*y*z 
                    - 7*y + 12*z**2 - 14*z + 3),
            ])
        elif i == 26:
            phi = -x*z*(4*x - 1)*(32*x + 32*y + 32*z - 32)
            dphi = np.array([
                -32*z*(8*x*y - y - z - 10*x + 8*x*z + 12*x**2 + 1),
                -32*x*z*(4*x - 1),
                -32*x*(4*x - 1)*(x + y + 2*z - 1),
            ])
        elif i == 27:
            phi = -x*z*(4*z - 1)*(32*x + 32*y + 32*z - 32)
            dphi = np.array([
                -32*z*(4*z - 1)*(2*x + y + z - 1),
                -32*x*z*(4*z - 1),
                -32*x*(8*x*z - y - 10*z - x + 8*y*z + 12*z**2 + 1),
            ])
        elif i == 28:
            phi = y*z*(4*x + 4*y + 4*z - 3)*(32*x + 32*y + 32*z - 32)
            dphi = np.array([
                32*y*z*(8*x + 8*y + 8*z - 7),
                32*z*(4*x**2 + 16*x*y + 8*x*z - 7*x + 12*y**2 + 16*y*z 
                    - 14*y + 4*z**2 - 7*z + 3),
                32*y*(4*x**2 + 8*x*y + 16*x*z - 7*x + 4*y**2 + 16*y*z 
                    - 7*y + 12*z**2 - 14*z + 3),
            ])
        elif i == 29:
            phi = -y*z*(4*y - 1)*(32*x + 32*y + 32*z - 32)
            dphi = np.array([
                -32*y*z*(4*y - 1),
                -32*z*(8*x*y - 10*y - z - x + 8*y*z + 12*y**2 + 1),
                -32*y*(4*y - 1)*(x + y + 2*z - 1),
            ])
        elif i == 30:
            phi = -y*z*(4*z - 1)*(32*x + 32*y + 32*z - 32)
            dphi = np.array([
                -32*y*z*(4*z - 1),
                -32*z*(4*z - 1)*(x + 2*y + z - 1),
                -32*y*(8*x*z - y - 10*z - x + 8*y*z + 12*z**2 + 1),
            ])
        elif i == 31:
            phi = 32*x*y*z*(4*x - 1)
            dphi = np.array([
                32*y*z*(8*x - 1),
                32*x*z*(4*x - 1),
                32*x*y*(4*x - 1),
            ])
        elif i == 32:
            phi = 32*x*y*z*(4*y - 1)
            dphi = np.array([
                32*y*z*(4*y - 1),
                32*x*z*(8*y - 1),
                32*x*y*(4*y - 1),
            ])
        elif i == 33:
            phi = 32*x*y*z*(4*z - 1)
            dphi = np.array([
                32*y*z*(4*z - 1),
                32*x*z*(4*z - 1),
                32*x*y*(8*z - 1),
            ])
        elif i == 34:
            phi = -x*y*z*(256*x + 256*y + 256*z - 256)
            dphi = np.array([
                -256*y*z*(2*x + y + z - 1),
                -256*x*z*(x + 2*y + z - 1),
                -256*x*y*(x + y + 2*z - 1),
            ])
        else:
            self._index_error()

        return phi, dphi

# solve Laplace eigenvalue problem on uniform meshes
$\Delta u = \lambda u $, in $\Omega = (0,1)^3$
and
$u = 0$, on $\partial\Omega.$
The first eigenvalue $\lambda_1=3\pi^2\approx29.608813203268074$

# Using p4 elements uniformly

In [12]:
from skfem import *
from skfem.models.poisson import laplace, mass
import numpy as np


for itr in range(6):
  #p = np.linspace(0, 1, itr+2)
  #m = MeshTet.init_tensor(*(p,) * 3)
  p1 = np.linspace(0, 5, itr+2)
  m1 = MeshTet.init_tensor(*(p1,) * 3)
  m2 = m1.mirrored([0,0,1])
  m3 = m1.mirrored([0,1,0])
  m4 = m1.mirrored([1,0,0])
  m5 = m1.mirrored([0,1,1])
  m6 = m1.mirrored([1,0,1])
  m7 = m1.mirrored([1,1,0])
  m8 = m7.mirrored([0,0,1])
  m = m8+m2+m3+m4+m5+m6+m7
  e = ElementTetP4()
  basis = Basis(m, e) 
  K = asm(laplace, basis)
  M = asm(mass, basis)
  D=basis.get_dofs()
  Lh,eigen_uh = solve(*condense(0.5*K, M, D=D), solver=solver_eigen_scipy_sym(sigma=0,k=1))
  Dof= K.shape[0]
  error = np.abs(Lh-0.2106412625)
  print("{},{},{},{}".format(itr,Dof,Lh,error))


0,665,[0.2131283],[0.00248703]
1,4401,[0.211695],[0.00105374]
2,13897,[0.21122613],[0.00058487]


Transforming over 1000 vertices to C_CONTIGUOUS.
Transforming over 1000 vertices to C_CONTIGUOUS.


3,31841,[0.21102614],[0.00038487]


Transforming over 1000 vertices to C_CONTIGUOUS.
Transforming over 1000 vertices to C_CONTIGUOUS.
Transforming over 1000 vertices to C_CONTIGUOUS.
Transforming over 1000 vertices to C_CONTIGUOUS.
Transforming over 1000 vertices to C_CONTIGUOUS.


4,60921,[0.21092005],[0.00027879]
5,103825,[0.21085582],[0.00021455]


# solve Laplace eigenvalue problem on the adaptive meshes
$\Delta u = \lambda u $, in $\Omega = (0,1)^3$

and
$u = 0$, on $\partial\Omega.$

The first eigenvalue $\lambda_1=3\pi^2\approx29.608813203268074$

# Using p2 elmements adaptively

In [ ]:
from skfem import *
from skfem.models.poisson import laplace, mass
from skfem.helpers import grad
import numpy as np

p = np.linspace(0, 1, 6)
m = MeshTet.init_tensor(*(p,) * 3)
e = ElementTetP2()


def eval_estimator(m, eigfvh, eigv):    
    # interior residual
    basis = Basis(m, e)

    grad_basis = basis.with_element(ElementVector(ElementDG(ElementTetP1()))) 
    w = {'grad_v': grad_basis.project(basis.interpolate(eigfvh).grad) }
    vh = basis.interpolate(eigfvh)

    @Functional 
    def interior_residual_vh(w): 
        h = w.h
        x, y,z = w.x
        (uxx, uxy,uxz), (uyx, uyy,uyz), (uzx, uzy,uzz) = w['grad_v'].grad 
        return h ** 2 * (uxx + uyy+uzz + eigv*vh) ** 2 

    eta_K_vh = interior_residual_vh.elemental(grad_basis, **w) 


    # facet jump
    fbasis = [InteriorFacetBasis(m, e, side=i) for i in [0, 1]]
    # jump of vh
    w = {'v' + str(i + 1): fbasis[i].interpolate(eigfvh) for i in [0, 1]}
    
    @Functional
    def edge_jump_vh(w):
        h = w.h
        n = w.n
        dw1 = grad(w['v1'])
        dw2 = grad(w['v2'])
        return h * ((dw1[0] - dw2[0]) * n[0] +
                    (dw1[1] - dw2[1]) * n[1] +
                    (dw1[2] - dw2[2]) * n[2]) ** 2

    eta_E_vh = edge_jump_vh.elemental(fbasis[0], **w)
    tmp_vh = np.zeros(m.facets.shape[1])
    np.add.at(tmp_vh, fbasis[0].find, eta_E_vh)
    eta_E_vh = np.sum(.5 * tmp_vh[m.t2f], axis=0)
    
    return eta_K_vh + eta_E_vh


for itr in reversed(range(12)):
        
    basis = Basis(m, e)
    
    K = asm(laplace, basis)
    M = asm(mass, basis)
    D=basis.get_dofs()
    Lh,eigen_uh = solve(*condense(K, M, D=D), solver=solver_eigen_scipy_sym(sigma=0,k=1))
    eigv = Lh[0]
    error = np.abs(eigv-3*np.pi**2)
    eigen_uh = eigen_uh.flatten()
    Dof = K.shape[0]
    print("{},{},{},{},{}".format(itr, m.param(), Dof, eigv, error))
        
    if itr > 0:
        m = m.refined(adaptive_theta(eval_estimator(m, eigen_uh,eigv)))

Transforming over 1000 elements to C_CONTIGUOUS.


11,0.34641016151377557,1331,29.706286231004054,0.09747302773597966


Transforming over 1000 elements to C_CONTIGUOUS.


10,0.3464101615137755,2027,29.64864481627044,0.039831613002366595
9,0.2828427124746191,2871,29.628758231498857,0.019945028230782924


Transforming over 1000 elements to C_CONTIGUOUS.


...

2,0.17320508075688779,21461,29.6101262149508,0.0013130116827255733


Transforming over 1000 vertices to C_CONTIGUOUS.
Transforming over 1000 elements to C_CONTIGUOUS.


1,0.14142135623730964,30749,29.609464822981806,0.0006516197137322877


Transforming over 1000 vertices to C_CONTIGUOUS.
Transforming over 1000 elements to C_CONTIGUOUS.


0,0.14142135623730964,34373,29.609343738098502,0.0005305348304283086


# Using p3 elements adaptively

In [ ]:
from skfem import *
from skfem.models.poisson import laplace, mass
from skfem.helpers import grad
import numpy as np

p = np.linspace(0, 1, 6)
m = MeshTet.init_tensor(*(p,) * 3)
e = ElementTetP3()


def eval_estimator(m, eigfvh, eigv):    
    # interior residual
    basis = Basis(m, e)

    grad_basis = basis.with_element(ElementVector(ElementDG(ElementTetP2()))) 
    w = {'grad_v': grad_basis.project(basis.interpolate(eigfvh).grad) }
    vh = basis.interpolate(eigfvh)

    @Functional 
    def interior_residual_vh(w): 
        h = w.h
        x, y,z = w.x
        (uxx, uxy,uxz), (uyx, uyy,uyz), (uzx, uzy,uzz) = w['grad_v'].grad 
        return h ** 2 * (uxx + uyy+uzz + eigv*vh) ** 2 

    eta_K_vh = interior_residual_vh.elemental(grad_basis, **w) 


    # facet jump
    fbasis = [InteriorFacetBasis(m, e, side=i) for i in [0, 1]]
    # jump of vh
    w = {'v' + str(i + 1): fbasis[i].interpolate(eigfvh) for i in [0, 1]}
    
    @Functional
    def edge_jump_vh(w):
        h = w.h
        n = w.n
        dw1 = grad(w['v1'])
        dw2 = grad(w['v2'])
        return h * ((dw1[0] - dw2[0]) * n[0] +
                    (dw1[1] - dw2[1]) * n[1] +
                    (dw1[2] - dw2[2]) * n[2]) ** 2

    eta_E_vh = edge_jump_vh.elemental(fbasis[0], **w)
    tmp_vh = np.zeros(m.facets.shape[1])
    np.add.at(tmp_vh, fbasis[0].find, eta_E_vh)
    eta_E_vh = np.sum(.5 * tmp_vh[m.t2f], axis=0)
    
    return eta_K_vh + eta_E_vh


for itr in reversed(range(12)):
        
    basis = Basis(m, e)
    
    K = asm(laplace, basis)
    M = asm(mass, basis)
    D=basis.get_dofs()
    Lh,eigen_uh = solve(*condense(K, M, D=D), solver=solver_eigen_scipy_sym(sigma=0,k=1))
    eigv = Lh[0]
    error = np.abs(eigv-3*np.pi**2)
    eigen_uh = eigen_uh.flatten()
    Dof = K.shape[0]
    print("{},{},{},{},{}".format(itr, m.param(), Dof, eigv, error))
        
    if itr > 0:
        m = m.refined(adaptive_theta(eval_estimator(m, eigen_uh,eigv)))

11,0.34641016151377557,4096,29.610201448711088,0.001388245443013858


Transforming over 1000 elements to C_CONTIGUOUS.


10,0.34641016151377557,5743,34.443115113394015,4.834301910125941


Transforming over 1000 elements to C_CONTIGUOUS.


9,0.3464101615137755,6409,35.80059383901864,6.191780635750565


...

2,0.3464101615137755,22972,38.8688762403918,9.260063037123725


Transforming over 1000 vertices to C_CONTIGUOUS.
Transforming over 1000 elements to C_CONTIGUOUS.


1,0.3464101615137755,26455,38.66783041160825,9.059017208340176


Transforming over 1000 vertices to C_CONTIGUOUS.
Transforming over 1000 elements to C_CONTIGUOUS.


0,0.34641016151377535,37546,38.77234628275499,9.163533079486914


# Using p4 elements adaptively

In [ ]:
from skfem import *
from skfem.models.poisson import laplace, mass
from skfem.helpers import grad
import numpy as np

p = np.linspace(0, 1, 6)
m = MeshTet.init_tensor(*(p,) * 3)
e = ElementTetP4()


def eval_estimator(m, eigfvh, eigv):    
    # interior residual
    basis = Basis(m, e)

    grad_basis = basis.with_element(ElementVector(ElementDG(ElementTetP3()))) 
    w = {'grad_v': grad_basis.project(basis.interpolate(eigfvh).grad) }
    vh = basis.interpolate(eigfvh)

    @Functional 
    def interior_residual_vh(w): 
        h = w.h
        x, y,z = w.x
        (uxx, uxy,uxz), (uyx, uyy,uyz), (uzx, uzy,uzz) = w['grad_v'].grad 
        return h ** 2 * (uxx + uyy+uzz + eigv*vh) ** 2 

    eta_K_vh = interior_residual_vh.elemental(grad_basis, **w) 


    # facet jump
    fbasis = [InteriorFacetBasis(m, e, side=i) for i in [0, 1]]
    # jump of vh
    w = {'v' + str(i + 1): fbasis[i].interpolate(eigfvh) for i in [0, 1]}
    
    @Functional
    def edge_jump_vh(w):
        h = w.h
        n = w.n
        dw1 = grad(w['v1'])
        dw2 = grad(w['v2'])
        return h * ((dw1[0] - dw2[0]) * n[0] +
                    (dw1[1] - dw2[1]) * n[1] +
                    (dw1[2] - dw2[2]) * n[2]) ** 2

    eta_E_vh = edge_jump_vh.elemental(fbasis[0], **w)
    tmp_vh = np.zeros(m.facets.shape[1])
    np.add.at(tmp_vh, fbasis[0].find, eta_E_vh)
    eta_E_vh = np.sum(.5 * tmp_vh[m.t2f], axis=0)
    
    return eta_K_vh + eta_E_vh


for itr in reversed(range(6)):
        
    basis = Basis(m, e)
    
    K = asm(laplace, basis)
    M = asm(mass, basis)
    D=basis.get_dofs()
    Lh,eigen_uh = solve(*condense(K, M, D=D), solver=solver_eigen_scipy_sym(sigma=0,k=1))
    eigv = Lh[0]
    error = np.abs(eigv-3*np.pi**2)
    eigen_uh = eigen_uh.flatten()
    Dof = K.shape[0]
    print("{},{},{},{},{}".format(itr, m.param(), Dof, eigv, error))
        
    if itr > 0:
        m = m.refined(adaptive_theta(eval_estimator(m, eigen_uh,eigv)))

5,0.34641016151377557,9261,29.608825237876108,1.2034608033673067e-05


Transforming over 1000 elements to C_CONTIGUOUS.


4,0.34641016151377557,13997,39.87218445498515,10.263371251717079


Transforming over 1000 elements to C_CONTIGUOUS.


3,0.34641016151377557,17621,40.68697153428219,11.07815833101412


...

2,0.34641016151377557,21785,39.92128229332504,10.312469090056968


Transforming over 1000 elements to C_CONTIGUOUS.


1,0.34641016151377557,26881,38.67292861657085,9.064115413302776


Transforming over 1000 elements to C_CONTIGUOUS.


0,0.34641016151377557,45685,39.39686921698523,9.788056013717156
